Neste notebook será avaliado o quão boa foi a vetorização das Keywords. Para tal usamos dois conjuntos de teste composto por três colunas. No primeiro conjuto usamos as próprias keywords, as duas primeiras colunas contém keywords que foram extraídas de um mesmo artigo, já a terceira coluna tem um keyword aleatória. Espera-se que os vetores da primeira e segunda coluna estejam mais próximo entre si do que do vetor da keyword aleatória. O segundo conjunto de teste usa a mesma abordagem mas ao invés de usar keywords usa os autores dos artigos.

Importando bibliotecas

In [1]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from scipy import spatial

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Lendo os arquivos

In [2]:
# Keyword Vectors
Keyword_vectors = pd.read_json("Elsevier_abstract_complete_text - Keywords.json")
# Keyword Test Corpus
Test_Corpus_keywords = pd.read_csv("Test_Corpus_keywords.csv", index_col=0)
# Creator Vectors
Creator_vectors = pd.read_json("Elsevier_abstract_complete_text - Creators.json")
# Creator Test Corpus
Test_Corpus_creator = pd.read_csv("Test_Corpus_creator.csv", index_col=0)
# Modelo Doc2Vec
doc2vec = Doc2Vec.load("AbstractKG_doc2vec_model")

In [3]:
# Vetores das Keywords
Creator_vectors.head()

,Abstract,Creator,Doc2Vec
0,"[detailed, experimental, data, including, temp...","kontogeorgis, georgios m.","[1.2801563740000002, 4.2987875938, -1.17563080..."
1,"[presently, upwelling, is, of, major, importan...","sinninghe damsté, jaap s.","[3.9878189564, 0.2559892535, 5.0495486259, -4...."
10,"[mineralogical, geochemical, structural, and, ...","cathelineau, michel","[-2.7941410542, -2.6100623607999998, 4.5452384..."
100,"[bench, top, nuclear, magnetic, resonance, nmr...","may, eric f.","[1.6743534803, 1.9385647773999999, 2.899723053..."
1000,"[combined, pb, and, lu, hf, isotope, data, fro...","laurent, oscar","[-5.2161836624, -3.8296308517, 4.0542969704, -..."


In [4]:
Test_Corpus_keywords.head()

,Keyword_1,Keyword_2,Random_keyword
0,neogene,paleoecology,climate
1,eastern mediterranean,speleothems,optimisation
2,eastern mediterranean,paleoclimate,geophysics
3,speleothems,paleoclimate,taiwan
4,são francisco craton,u–pb dating,sediment transport


In [5]:
# Triplas de Keywords para avaliação
Test_Corpus_creator.head()

,Creator_1,Creator_2,Random_Creator
0,"bar-matthews, miryam","ayalon, avner","di primio, rolando"
1,"xing, lida","lockley, martin g.","feist, stephen w."
2,"xing, lida","klein, hendrik","burnham, alan k."
3,"lockley, martin g.","klein, hendrik","palomera, isabel"
4,"stenby, erling h.","thomsen, kaj","borgomano, jean"


## Keywords

Unindo os corpus de teste com os vetores das keywords

In [6]:
# Criando arrays para os vetores
vector_1 = []
vector_2 = []
random_vector = []

# Para cada linha do corpus de teste identificar os vetores para cada keyword
for index, line in Test_Corpus_keywords.iterrows():
    vector_1.append(Keyword_vectors[Keyword_vectors["Keywords"]==line[0]]["Doc2Vec"].values[0])
    vector_2.append(Keyword_vectors[Keyword_vectors["Keywords"]==line[1]]["Doc2Vec"].values[0])
    random_vector.append(Keyword_vectors[Keyword_vectors["Keywords"]==line[2]]["Doc2Vec"].values[0])

In [7]:
# Incorporar os vetores no DataFrame "test_corpus"
Test_Corpus_keywords["vector_1"] = vector_1
Test_Corpus_keywords["vector_2"] = vector_2
Test_Corpus_keywords["random_vector"] = random_vector

In [8]:
# Incorporar as distâncias coseno entre os vetores
Test_Corpus_keywords["CosDist 1-2"] = Test_Corpus_keywords[["vector_1",
                                                            "vector_2"]].apply(lambda x: spatial.
                                                                               distance.
                                                                               cosine(*x),
                                                                               axis=1)
Test_Corpus_keywords["CosDist 1-Random"] = Test_Corpus_keywords[["vector_1",
                                                                 "random_vector"]].apply(lambda x: spatial.
                                                                                         distance.
                                                                                         cosine(*x),
                                                                                         axis=1)
Test_Corpus_keywords["CosDist 2-Random"] = Test_Corpus_keywords[["vector_2",
                                                                 "random_vector"]].apply(lambda x: spatial.
                                                                                         distance.
                                                                                         cosine(*x),
                                                                                         axis=1)

In [9]:
# O score é média entre as diferenças entre CostDist 1 - Random e CosDist 1 - 2, CostDist 2 - Random e CosDist 1 - 2
Test_Corpus_keywords["Score"] = ((Test_Corpus_keywords["CosDist 1-Random"] - 
                                  Test_Corpus_keywords["CosDist 1-2"]) +
                                 (Test_Corpus_keywords["CosDist 2-Random"] - 
                                  Test_Corpus_keywords["CosDist 1-2"])) / 2

In [10]:
# O Score_bool é True quando CosDist 1-2 é maior que as distancias para o vetor aleatório. 
# A métrica é dado pelo percentual de True no total
Test_Corpus_keywords["Score_bool"] = ((Test_Corpus_keywords["CosDist 1-Random"] >
                                       Test_Corpus_keywords["CosDist 1-2"]) &
                                      (Test_Corpus_keywords["CosDist 2-Random"] > 
                                       Test_Corpus_keywords["CosDist 1-2"]))

Calculando o score para as keywords

In [11]:
keyword_score = Test_Corpus_keywords["Score"].mean()
keyword_score_bool = (Test_Corpus_keywords["Score_bool"].sum() /
                      len(Test_Corpus_keywords["Score_bool"]))

print ("O Keyword Score foi: ", keyword_score)
print ("O Keyword Score_bool foi: ", keyword_score_bool)

O Keyword Score foi:  0.6281530463016096
O Keyword Score_bool foi:  0.7061235977644646


## Autores

Unindo os corpus de teste com os vetores dos autores

In [12]:
# Criando arrays para os vetores
vector_1 = []
vector_2 = []
random_vector = []

# Para cada linha do corpus de teste identificar os vetores para cada autor

for index, line in Test_Corpus_creator.iterrows():
    try:        
        vector_1.append(Creator_vectors[Creator_vectors["Creator"]==line[0]]["Doc2Vec"].values[0])
    except:
        vector_1.append('branco')
    
    try:
        vector_2.append(Creator_vectors[Creator_vectors["Creator"]==line[1]]["Doc2Vec"].values[0])
    except:
        vector_2.append('branco')
    
    try:
        random_vector.append(Creator_vectors[Creator_vectors["Creator"]==line[2]]["Doc2Vec"].values[0])
    except:
        random_vector.append('branco')

In [13]:
# Incorporar os vetores no DataFrame "Test_Corpus_creator"
Test_Corpus_creator["vector_1"] = vector_1
Test_Corpus_creator["vector_2"] = vector_2
Test_Corpus_creator["random_vector"] = random_vector

In [14]:
# Retirando os vetores que retornaram em branco
Test_Corpus_creator = Test_Corpus_creator[(Test_Corpus_creator["vector_1"]!='branco') &
                                          (Test_Corpus_creator["vector_2"]!='branco') & 
                                          (Test_Corpus_creator["random_vector"]!='branco')]

In [15]:
# Incorporar as distâncias coseno entre os vetores
Test_Corpus_creator["CosDist 1-2"] = Test_Corpus_creator[["vector_1",
                                                          "vector_2"]].apply(lambda x: spatial.
                                                                             distance.
                                                                             cosine(*x),
                                                                             axis=1)
Test_Corpus_creator["CosDist 1-Random"] = Test_Corpus_creator[["vector_1",
                                                               "random_vector"]].apply(lambda x: spatial.
                                                                                       distance.
                                                                                       cosine(*x),
                                                                                       axis=1)
Test_Corpus_creator["CosDist 2-Random"] = Test_Corpus_creator[["vector_2",
                                                               "random_vector"]].apply(lambda x: spatial.
                                                                                       distance.
                                                                                       cosine(*x),
                                                                                       axis=1)

In [16]:
# O score é média entre as diferenças entre CostDist 1 - Random e CosDist 1 - 2, CostDist 2 - Random e CosDist 1 - 2
Test_Corpus_creator["Score"] = ((Test_Corpus_creator["CosDist 1-Random"] - 
                                 Test_Corpus_creator["CosDist 1-2"]) +
                                (Test_Corpus_creator["CosDist 2-Random"] - 
                                 Test_Corpus_creator["CosDist 1-2"])) / 2

In [17]:
# O Score_bool é True quando CosDist 1-2 é maior que as distancias para o vetor aleatório. 
# A métrica é dado pelo percentual de True no total
Test_Corpus_creator["Score_bool"] = ((Test_Corpus_creator["CosDist 1-Random"] >
                                      Test_Corpus_creator["CosDist 1-2"]) &
                                     (Test_Corpus_creator["CosDist 2-Random"] > 
                                      Test_Corpus_creator["CosDist 1-2"]))

Calculando o score para os autores

In [18]:
creator_score = Test_Corpus_creator["Score"].mean()
creator_score_bool = (Test_Corpus_creator["Score_bool"].sum() /
                      len(Test_Corpus_creator["Score_bool"]))

print ("O Creator Score foi: ", creator_score)
print ("O Creator Score_bool foi: ", creator_score_bool)

O Creator Score foi:  0.898249460249788
O Creator Score_bool foi:  0.9145847400405132


## Score Global

Consideramos o score global como a média dos scores das keywords e dos autores

In [19]:
print ("O Global Score foi: ", (creator_score + keyword_score) / 2)
print ("O Global Score_bool foi: ", (creator_score_bool + keyword_score_bool) / 2)

O Global Score foi:  0.7632012532756989
O Global Score_bool foi:  0.8103541689024889
